In [1]:
from data_processing import csv_data, write_to_json_file
from forecasting import arima_estimation, sarima_estimation
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import time
import json

In [2]:
dataClass = csv_data("../data/urgence3.csv")

c:\users\sofiane\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3347: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
start_d, start_m, start_y = 1, 1, 2019

In [4]:
path = "../data/predictions4/"

prev_week_load_file = "prev_week_load.json"
prediction_file_name = "predictions.json"

In [5]:
n_prev_days = 7
prev_week_load = dataClass.return_prev_days(start_y, start_m, start_d, n_prev_days = n_prev_days)
index = dataClass.find_start_day(start_y, start_m, start_d)

In [6]:
stop = 1
interval = 2
n_samples = 100
date = datetime(start_y, start_m, start_d, 0, 0)
dic_key = f"{start_y:02}-{start_m:02}-{start_d:02} 00:00"
prev_week_load_dict = {dic_key:prev_week_load}
prev_index = index
prev_week_load_dict[dic_key].append(0)

while(True and index >= 0):
    date += timedelta(minutes = interval)
    time.sleep(1)
    
    while(datetime.strptime(dataClass.dates[index], dataClass.dateFormat) < date):
        prev_week_load_dict[dic_key][-1] += 1
        index -= 1
    if(prev_index != index):
        write_to_json_file(path, prev_week_load_file, prev_week_load_dict)
        model = arima_estimation(prev_week_load_dict[dic_key][-n_samples:])
        print(dataClass.dates[index], model.aic)
        try:
            predictions = model.forecast(steps = 24)
            predictions_dict = {f"{start_y:02}-{start_m:02}-{start_d:02}": list(map(round, predictions))}
            write_to_json_file(path, prediction_file_name, predictions_dict)
        except:
            raise ValueError("Cannot forecast !!")
    
    prev_index = index
    if(date.minute == 0):
        prev_week_load_dict[dic_key].append(0)
    
#     print(index, dataClass.dates[index], date)

c:\users\sofiane\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


01/01/2019 00:14 404.1863568325818
01/01/2019 00:17 403.4143323749864
01/01/2019 00:55 402.83051715494474
01/01/2019 00:56 402.43631810908147
01/01/2019 00:59 402.2158587154583
01/01/2019 01:02 402.38619545105644
01/01/2019 01:06 402.6635556298335
01/01/2019 01:09 402.4319723845822
01/01/2019 01:32 402.3865592360323


KeyboardInterrupt: 